## 1. Load packages and data

In [1]:
# import packages
import os 

import re
from datetime import datetime

import pytz

import pandas as pd
import numpy as np

import ast

import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, udf, lit
from sparknlp.pretrained import PretrainedPipeline 


import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import emojis
from translate import Translator

import sparknlp


In [2]:
import sparknlp 

spark = sparknlp.start(m1=True)

print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

22/12/01 18:26:14 WARN Utils: Your hostname, MacBook-Pro-van-Wouter.local resolves to a loopback address: 127.0.0.1; using 192.168.0.134 instead (on interface en0)
22/12/01 18:26:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/wouterdewitte/.ivy2/cache
The jars for the packages stored in: /Users/wouterdewitte/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-m1_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c4a394df-3147-44d5-9ba3-ff3a8c6a8f32;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-m1_2.12;4.2.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found

22/12/01 18:26:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version
Apache Spark version


'3.3.1'

In [3]:
# import findspark
import findspark

# initialize findspark with spark directory

#ALWAYS HAVE TO BE CHANGED 
findspark.init("/Users/wouterdewitte/spark/")

# import pyspark
import pyspark
# create spark context
#sc = pyspark.SparkContext()
# create spark session 
#spark = pyspark.sql.SparkSession(sc)

In [4]:
#set this path to your path, for some reason I have an error 
#reading in all the files
#path_json = ".././../data/Topic_vegan/*.json"

# use this if you want all the tweet files, but this is usually too large
#df_json = spark.read.json(path_json)

In [5]:
list_brands = ["healthyfood",
               "healthylifestyle",
               "_vegan_",
               "keto",
               "ketodiet",
               "ketolifestyle",
               "veganism",
               "vegetarian"]
from re import search



data_dir = ".././../data/Topic_vegan/"
tweet_files = [os.path.join(data_dir, obs) for obs in os.listdir(data_dir)]


#filter on correct files via keyword
files_brand = [file for file in tweet_files if (file.find(list_brands[2]) != -1)]
files_brand               
               
df_json = spark.read.option("multiline","true").json(files_brand)  
df_json.count()

22/12/01 18:26:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


1595676

In [6]:
# select interesting features
import pyspark.sql.functions as F

df = df_json.select(F.col("user.name"),
                    F.col("user.screen_name"),
                    F.col("created_at"), 
                    F.col("full_text"))
df.limit(10).toPandas()

,name,screen_name,created_at,full_text
0,şᑭ𝐄cιⒺ𝔰ａＭ𝔲𝐬ｅᵈ 🦇🌳🐒🐴🐛,speciesamused,Tue Sep 13 22:32:32 +0000 2022,RT @animalsavemvmt: Do you see us? Will you he...
1,Camuka 🇹🇷,Zomorok,Tue Sep 13 22:32:26 +0000 2022,RT @angie_karan: #vegan \n for the animals...
2,Michael Belton,38Belton,Tue Sep 13 22:32:26 +0000 2022,RT @MyVegan_Reach: Cows are forcibly impregnat...
3,Stare Decisis Ⓥ,do_nothing_dem,Tue Sep 13 22:32:16 +0000 2022,RT @angie_karan: #vegan \n for the animals...
4,mally,mizzishyde,Tue Sep 13 22:32:09 +0000 2022,RT @angie_karan: #vegan \n for the animals...
5,hiedra-vegan,vegan02131055,Tue Sep 13 22:32:07 +0000 2022,"RT @Dodo_Tribe: Jared Leto - ""No More Pus"" #G..."
6,RadioFreeKrsna,JFave5,Tue Sep 13 22:31:54 +0000 2022,RT @veganrecipebowl: This recipe is also #vega...
7,VerdeVetriolo@gingerzoerescueranch,VerdeVetriolo,Tue Sep 13 22:31:54 +0000 2022,RT @animalsavemvmt: Do you see us? Will you he...
8,Lightspeed,LightspeedSteps,Tue Sep 13 22:31:49 +0000 2022,RT @DanielleAnd15: #vegan https://t.co/a1eSrYlq23
9,kaz5thlife,kaz6thlife,Tue Sep 13 22:31:49 +0000 2022,刺激的だけど‥動物好きなら、必ず理解する画像です。よね❗️#vegan の気持ちは良く理解で...


## 2. Preprocess Data

UDF’s are used to extend the functions of the framework and re-use these functions on multiple DataFrame’s.

In [7]:
# https://developer.twitter.com/en/docs/twitter-ads-api/timezones
# function to convert Twitter date string format
def getDate(date):
    if date is not None:
        return str(datetime.strptime(date,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None

# UDF declaration
date_udf = F.udf(getDate, StringType())

# apply udf
df = df.withColumn('post_created_at', F.to_utc_timestamp(date_udf("created_at"), "UTC"))

In [8]:
#drop duplicates and retweets 
df = df.filter(~F.col("full_text").startswith("RT"))\
                        .drop_duplicates()
#sorting such when dropping later we only keep the most recent post 
df = df.sort("post_created_at", ascending=False)
#removing spam accounts 
df = df.drop_duplicates(["full_text", "screen_name"])

#df.printSchema()
#df.count() #1340938

In [9]:
# define function to count hashtags
def get_hashtags(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "#" in word:
            counter += 1
    return(counter) 

# define function to count mentions
def get_mentions(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "@" in word:
            counter += 1
    return(counter)

# define function to count exclamation marks
def get_exclamation_marks(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "!" in word:
            counter += 1
    return(counter)

# define function to count number of emojis used
import emojis
def emoji_counter(text):
    nr_emojis = emojis.count(text)
    return(nr_emojis)
# register functions as udf
get_hashtags_UDF = F.udf(get_hashtags, IntegerType())
get_mentions_UDF = F.udf(get_mentions, IntegerType())
get_exclamation_marks_UDF = F.udf(get_exclamation_marks, IntegerType())
emoji_counter_udf = F.udf(emoji_counter, IntegerType())


In [10]:
twitter_df = df.withColumn("emoji_count", emoji_counter_udf("full_text")) \
                            .withColumn("text_tokenized", F.split("full_text", " ")) \
                            .withColumn("num_words", F.size("text_tokenized")) \
                            .withColumn("num_hashtags", get_hashtags_UDF("text_tokenized")) \
                            .withColumn("num_mentions", get_mentions_UDF("text_tokenized")) \
                            .withColumn("num_exclamation_marks", get_exclamation_marks_UDF("text_tokenized"))
twitter_df.show()

+--------------------+---------------+--------------------+--------------------+-------------------+-----------+--------------------+---------+------------+------------+---------------------+
|                name|    screen_name|          created_at|           full_text|    post_created_at|emoji_count|      text_tokenized|num_words|num_hashtags|num_mentions|num_exclamation_marks|
+--------------------+---------------+--------------------+--------------------+-------------------+-----------+--------------------+---------+------------+------------+---------------------+
| Follow the Vegans Ⓥ|  vegan_v_vegan|Sat May 14 00:55:...|!\n#vegan #GoVega...|2022-05-14 00:55:33|          0|[!\n#vegan, #GoVe...|        4|           3|           0|                    1|
|🌱Veg-In-Out Mark...| veginoutmarket|Sat Jan 15 07:17:...|! We will be open...|2022-01-15 07:17:18|          0|[!, We, will, be,...|       35|          21|           0|                    3|
|         Mix 93.8 FM|       Mix938FM|Wed

In [1]:
# define function to clean text
def clean_text(string):
    
    # define numbers
    NUMBERS = '0123456789'
    PUNCT = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    
    # convert text to lower case
    cleaned_string = string.lower()
    
    # remove URLS
    cleaned_string = re.sub(r'http\S+', ' ', cleaned_string)
    
    # replace emojis by words
    cleaned_string = emojis.decode(cleaned_string)
    cleaned_string = cleaned_string.replace(":"," ").replace("_"," ")
    cleaned_string = ' '.join(cleaned_string.split())
    
    # remove numbers
    cleaned_string = "".join([char for char in cleaned_string if char not in NUMBERS])
    
    # remove punctuation
    cleaned_string = "".join([char for char in cleaned_string if char not in PUNCT])
    
    # remove words consisting out of one character (or less)
    cleaned_string = ' '.join([w for w in cleaned_string.split() if len(w) > 1])

    # return
    return(cleaned_string) 
clean_text_udf = F.udf(clean_text, StringType())


NameError: name 'F' is not defined

In [12]:
twitter_df = df.withColumn("text", clean_text_udf(F.col("full_text")))

In [13]:
#from sparknlp.pretrained import PretrainedPipeline 
#pipeline = PretrainedPipeline("translate_mul_en", lang = "xx")

In [14]:
#twitter_df = pipeline.transform(twitter_df)

In [15]:
from googletrans import Translator

translator = Translator() 
translation = translator.translate("test", dest='en')

def translate(tweet):
    if len(tweet)!=0:
        translator = Translator() 
        translation = translator.translate(tweet, dest = 'en')
        return(translation.text)
    return None

translate_udf = F.udf(translate, StringType())

twitter_df = twitter_df.withColumn("translation", translate_udf("text"))

twitter_df.show()

22/12/01 18:28:57 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 29 (TID 2111): Attempting to kill Python Worker
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|                name|    screen_name|          created_at|           full_text|    post_created_at|                text|         translation|
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+
| Follow the Vegans Ⓥ|  vegan_v_vegan|Sat May 14 00:55:...|!\n#vegan #GoVega...|2022-05-14 00:55:33| vegan govegan dairy| vegan govegan dairy|
|🌱Veg-In-Out Mark...| veginoutmarket|Sat Jan 15 07:17:...|! We will be open...|2022-01-15 07:17:18|we will be open a...|we will be open a...|
|         Mix 93.8 FM|       Mix938FM|Wed Sep 07 09:11:...|!! Daily Updates ...|2022-09-07 09:11:40|daily updates tas...|daily updates tas...|
| 

In [16]:
twitter_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- full_text: string (nullable = true)
 |-- post_created_at: timestamp (nullable = true)
 |-- text: string (nullable = true)
 |-- translation: string (nullable = true)



In [17]:
#twitter_df = twitter_df.withColumn("translation", twitter_df.translation.result)

In [18]:
#twitter_df = twitter_df.drop("document","sentence")

In [19]:
twitter_df.show()

22/12/01 18:30:16 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 41 (TID 2264): Attempting to kill Python Worker
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|                name|    screen_name|          created_at|           full_text|    post_created_at|                text|         translation|
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+
| Follow the Vegans Ⓥ|  vegan_v_vegan|Sat May 14 00:55:...|!\n#vegan #GoVega...|2022-05-14 00:55:33| vegan govegan dairy| vegan govegan dairy|
|🌱Veg-In-Out Mark...| veginoutmarket|Sat Jan 15 07:17:...|! We will be open...|2022-01-15 07:17:18|we will be open a...|we will be open a...|
|         Mix 93.8 FM|       Mix938FM|Wed Sep 07 09:11:...|!! Daily Updates ...|2022-09-07 09:11:40|daily updates tas...|daily updates tas...|
| 

# 3. Topic Modelling

https://github.com/maobedkova/TopicModelling_PySpark_SparkNLP/blob/master/Topic_Modelling_with_PySpark_and_Spark_NLP.ipynb

https://www.johnsnowlabs.com/spark-nlp/


## 3.1 Spark NLP pipeline

### 3.1.1 Basic NLP pipeline

DocumentAssembler converts data into Spark NLP annotation format that can be used by Spark NLP annotators. Prepares data into a format that is processable by Spark NLP

In [20]:
from sparknlp.base import DocumentAssembler
documentAssembler = DocumentAssembler() \
                .setInputCol("translation") \
                .setOutputCol('document')

Next, we tokenize the data with Tokenizer. Tokenizes raw text in document type columns into TokenizedSentence. Tokenization is used in natural language processing to split paragraphs and sentences into smaller units that can be more easily assigned meaning.

In [21]:
from sparknlp.annotator import Tokenizer
tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('tokenized')

We clean out the data and lower it with Normalizer. Annotator that cleans out tokens. Requires stems, hence tokens. Removes all dirty characters from text following a regex pattern and transforms words based on a provided dictionary.



In [22]:
from sparknlp.annotator import Normalizer
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

We are going to lemmatize our text with pretrained lemming model provided by Spark NLP. We can access this model with LemmatizerModel.
Class to find lemmas out of words with the objective of returning a base dictionary word. Retrieves the significant part of a word. A dictionary of predefined lemmas must be provided with . The dictionary can be set as a delimited text file. Pretrained models can be loaded with .

In [23]:
from sparknlp.annotator import LemmatizerModel
lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemmatized')

lemma_antbnc download started this may take some time.
Approximate size to download 907,6 KB
[ | ]lemma_antbnc download started this may take some time.
Approximate size to download 907,6 KB
Download done! Loading the resource.
[OK!]


Spark NLP doesn't provide stop word list, hence, we will use nltk package to download stop words for English.

In [24]:
from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')

In [25]:
from sparknlp.annotator import StopWordsCleaner
stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('unigrams') \
     .setStopWords(eng_stopwords)

In addition to unigrams, it is good to use n-grams for topic modelling as well since they help to better refine topics. We can get n-grams with NGramGenerator in Spark NLP. N-grams try to predict which word will have the highest probability of appearing with the other words

In [26]:
from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator() \
    .setInputCols(['lemmatized']) \
    .setOutputCol('ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

We already have our basic NLP pipeline for topic modelling with all necessary steps. However, let's use POS tagger in order to improve our processed data for topic modelling even more with POS tagged data later. For this, we are going to use pretrained POS tagging model provided by Spark NLP. We can access the model with PerceptronModel. Trains an averaged Perceptron model to tag words part-of-speech. Sets a POS tag to each word within a sentence.

In [27]:
from sparknlp.annotator import PerceptronModel
pos_tagger = PerceptronModel.pretrained('pos_anc') \
     .setInputCols(['document', 'lemmatized']) \
     .setOutputCol('pos')

pos_anc download started this may take some time.
Approximate size to download 3,9 MB
[ / ]pos_anc download started this may take some time.
Approximate size to download 3,9 MB
Download done! Loading the resource.


[OK!]


Now we have everything in Spark NLP annotation format. To be able to process the data further, we need to tranform data with Finisher. Converts annotation results into a format that easier to use. It is useful to extract the results from Spark NLP Pipelines. The Finisher outputs annotation(s) values into String.



In [28]:
from sparknlp.base import Finisher
finisher = Finisher() \
     .setInputCols(['unigrams', 'ngrams', 'pos'])

Now we are ready to input everything into a pipeline. Pipeline functionality is accessible with PySpark.

In [29]:
from pyspark.ml import Pipeline
pipeline = Pipeline() \
     .setStages([documentAssembler,
                 tokenizer,
                 normalizer,
                 lemmatizer,
                 stopwords_cleaner,
                 pos_tagger,
                 ngrammer,
                 finisher])

In [30]:
processed_review = pipeline.fit(twitter_df).transform(twitter_df)

In [31]:
processed_review.drop("full_text","text")

DataFrame[name: string, screen_name: string, created_at: string, post_created_at: timestamp, translation: string, finished_unigrams: array<string>, finished_ngrams: array<string>, finished_pos: array<string>]

In [32]:
processed_review.limit(5).show()

22/12/01 18:31:49 WARN PythonUDFRunner: Detected deadlock while completing task 4.0 in stage 59 (TID 2452): Attempting to kill Python Worker
22/12/01 18:31:49 WARN PythonUDFRunner: Detected deadlock while completing task 1.0 in stage 59 (TID 2449): Attempting to kill Python Worker


22/12/01 18:31:51 WARN PythonUDFRunner: Detected deadlock while completing task 7.0 in stage 59 (TID 2455): Attempting to kill Python Worker


22/12/01 18:31:53 WARN PythonUDFRunner: Detected deadlock while completing task 6.0 in stage 59 (TID 2454): Attempting to kill Python Worker
22/12/01 18:31:53 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 59 (TID 2448): Attempting to kill Python Worker


22/12/01 18:31:55 WARN PythonUDFRunner: Detected deadlock while completing task 3.0 in stage 59 (TID 2451): Attempting to kill Python Worker


22/12/01 18:31:57 WARN PythonUDFRunner: Detected deadlock while completing task 5.0 in stage 59 (TID 2453): Attempting to kill Python Worker


22/12/01 18:32:05 WARN PythonUDFRunner: Detected deadlock while completing task 2.0 in stage 59 (TID 2450): Attempting to kill Python Worker


22/12/01 18:32:41 WARN PythonUDFRunner: Detected deadlock while completing task 8.0 in stage 59 (TID 2456): Attempting to kill Python Worker
+--------------------+--------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                name|   screen_name|          created_at|           full_text|    post_created_at|                text|         translation|   finished_unigrams|     finished_ngrams|        finished_pos|
+--------------------+--------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| Follow the Vegans Ⓥ| vegan_v_vegan|Sat May 14 00:55:...|!\n#vegan #GoVega...|2022-05-14 00:55:33| vegan govegan dairy| vegan govegan dairy|[vegan, govegan, ...|[vegan, govegan, ...|        [NN, NN, NN]|
|🌱Veg-In-Out Mark...|veginoutmarket|Sat

### 3.1.2 Extended NLP pipeline

Up to now, we have our data in a form of unigrams that are lemmatized, with no stop words in there. I think it is a good idea to incorporate n-grams into our NLP pipeline. We obtained n-grams as one step of our pipeline but now n-grams are messy and have a lot of questionable combinations in there. To tackle this problem, let's filter out strange combinations of words in n-grams based on their POS tags. We can imagine a list of viable combinations like ADJ + NOUN so let's restrict our POS combinations in n-grams to this list. Plus, we can also exclude some POS tags from our unigrams to ensure that we don't use functional words for topic modelling (they can be partially covered by stop words but probably not fully).

Doing this POS-based filtering will significantly reduce the vocabulary size for topic modelling which will speed up the whole processing.

Let's start this processing. First, we need to join all our POS tags obtained previously.

In [33]:
from pyspark.sql import types as T

udf_join_arr = F.udf(lambda x: ' '.join(x), T.StringType())
processed_review  = processed_review.withColumn('finished_pos', udf_join_arr(F.col('finished_pos')))

Then we start another Spark NLP pipeline in order to get POS tag n-grams that correspond to word n-grams. We start with convertation into Spark NLP annotation format.

In [34]:
pos_documentAssembler = DocumentAssembler() \
     .setInputCol('finished_pos') \
     .setOutputCol('pos_document')

Then, we tokenize our POS tags.

In [35]:
pos_tokenizer = Tokenizer() \
     .setInputCols(['pos_document']) \
     .setOutputCol('pos')

And generate n-grams from them in the same way we did that for words.

In [36]:
pos_ngrammer = NGramGenerator() \
    .setInputCols(['pos']) \
    .setOutputCol('pos_ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

Lastly, we are ready to get POS tags ngrams with Finisher.

In [37]:
pos_finisher = Finisher() \
     .setInputCols(['pos', 'pos_ngrams'])

We create this new Spark NLP pipeline...

In [38]:
pos_pipeline = Pipeline() \
     .setStages([pos_documentAssembler,                  
                 pos_tokenizer,
                 pos_ngrammer,  
                 pos_finisher])

... and again fit it and transform the data.

In [39]:
processed_review = pos_pipeline.fit(processed_review).transform(processed_review)

Let's look what kind of data we have to operate with.

In [40]:
processed_review.columns

['name',
 'screen_name',
 'created_at',
 'full_text',
 'post_created_at',
 'text',
 'translation',
 'finished_unigrams',
 'finished_ngrams',
 'finished_pos',
 'finished_pos_ngrams']

And these are our word n-grams with their corresponding pos n-grams.

In [42]:
processed_review.select('finished_ngrams', 'finished_pos_ngrams').limit(5).show()

22/12/02 08:23:38 WARN PythonUDFRunner: Detected deadlock while completing task 6.0 in stage 76 (TID 2616): Attempting to kill Python Worker


22/12/02 08:23:48 WARN PythonUDFRunner: Detected deadlock while completing task 2.0 in stage 76 (TID 2612): Attempting to kill Python Worker
22/12/02 08:23:48 WARN PythonUDFRunner: Detected deadlock while completing task 5.0 in stage 76 (TID 2615): Attempting to kill Python Worker
22/12/02 08:23:48 WARN PythonUDFRunner: Detected deadlock while completing task 4.0 in stage 76 (TID 2614): Attempting to kill Python Worker
22/12/02 08:23:48 WARN PythonUDFRunner: Detected deadlock while completing task 1.0 in stage 76 (TID 2611): Attempting to kill Python Worker


22/12/02 08:23:50 WARN PythonUDFRunner: Detected deadlock while completing task 7.0 in stage 76 (TID 2617): Attempting to kill Python Worker


22/12/02 08:23:52 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 76 (TID 2610): Attempting to kill Python Worker
22/12/02 08:23:52 WARN PythonUDFRunner: Detected deadlock while completing task 3.0 in stage 76 (TID 2613): Attempting to kill Python Worker


22/12/02 08:26:48 WARN PythonUDFRunner: Detected deadlock while completing task 8.0 in stage 76 (TID 2618): Attempting to kill Python Worker
+--------------------+--------------------+
|     finished_ngrams| finished_pos_ngrams|
+--------------------+--------------------+
|[this, afternoon,...|[DT, NN, VBG, JJ,...|
|[sweet, potato, a...|[JJ, NN, CC, NN, ...|
|[creamy, mushroom...|[NN, NN, NN, NN, ...|
|[polypieter, vegg...|[NN, NN, VB, RB, ...|
|[enrich, with, na...|[NN, IN, JJ, NN, ...|
+--------------------+--------------------+



Now we are ready to filter out not useful for topic modelling analysis POS tags from our data. Let's create the function that does it for unigrams first. We create the custom Python function and then transform it to PySpark UDF to be used on Spark dataframe. WHAT ARE THE POS TAGSSSS

- NN is singular noun
- NNS is plural noun
- VB is verb
- VBP verb, present tense not 3rd person singular(wrap)
- JJ is an adjective (large)

In [43]:
def filter_pos(words, pos_tags):
    return [word for word, pos in zip(words, pos_tags) 
            if pos in ['JJ', 'NN', 'NNS', 'VB', 'VBP']]

udf_filter_pos = F.udf(filter_pos, T.ArrayType(T.StringType()))

Then, we apply this function on columns with unigrams and their POS tags to get filtered unigrams in a separate dataframe column.

In [44]:
processed_review = processed_review.withColumn('filtered_unigrams',
                                               udf_filter_pos(F.col('finished_unigrams'), 
                                                              F.col('finished_pos')))

That is how our filtered unigrams look like.

In [45]:
processed_review.select('filtered_unigrams').limit(5).show(truncate=90)

22/12/02 08:29:20 WARN PythonUDFRunner: Detected deadlock while completing task 2.0 in stage 93 (TID 2774): Attempting to kill Python Worker
22/12/02 08:29:20 WARN PythonUDFRunner: Detected deadlock while completing task 3.0 in stage 93 (TID 2775): Attempting to kill Python Worker


22/12/02 08:29:22 WARN PythonUDFRunner: Detected deadlock while completing task 5.0 in stage 93 (TID 2777): Attempting to kill Python Worker
22/12/02 08:29:22 WARN PythonUDFRunner: Detected deadlock while completing task 7.0 in stage 93 (TID 2779): Attempting to kill Python Worker
22/12/02 08:29:22 WARN PythonUDFRunner: Detected deadlock while completing task 1.0 in stage 93 (TID 2773): Attempting to kill Python Worker
22/12/02 08:29:22 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 93 (TID 2772): Attempting to kill Python Worker


22/12/02 08:29:24 WARN PythonUDFRunner: Detected deadlock while completing task 4.0 in stage 93 (TID 2776): Attempting to kill Python Worker
22/12/02 08:29:24 WARN PythonUDFRunner: Detected deadlock while completing task 6.0 in stage 93 (TID 2778): Attempting to kill Python Worker


22/12/02 08:31:58 WARN PythonUDFRunner: Detected deadlock while completing task 8.0 in stage 93 (TID 2780): Attempting to kill Python Worker
+------------------------------------------------------------------------------------------+
|                                                                         filtered_unigrams|
+------------------------------------------------------------------------------------------+
|[livestreaming, conversation, asefrid, book, govegan, livelikeagorilla, veganfortheanim...|
|                                   [sweet, potato, fritter, mint, dip, vegan, plant, base]|
|                           [creamy, mushroom, bucatini, vegan, veganforlife, forevervegan]|
|[polypieter, veggie, already, vegan, almost, impossible, get, hey, dont, ingredient, ve...|
|                             [enrich, origin, ingredient, certify, vegan, wink, fragrance]|
+------------------------------------------------------------------------------------------+



It is time to filter out improper POS combinations of n-grams. We create the custom function in the same manner as before. Since we deal with bi- and trigrams, we need to restrict tags for both.

In [46]:
def filter_pos_combs(words, pos_tags):
    return [word for word, pos in zip(words, pos_tags) 
            if (len(pos.split('_')) == 2 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS']) \
            or (len(pos.split('_')) == 3 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                  pos.split('_')[2] in ['NN', 'NNS'])]
    
udf_filter_pos_combs = F.udf(filter_pos_combs, T.ArrayType(T.StringType()))

And we call the function on word and POS n-grams.

In [47]:
processed_review = processed_review.withColumn('filtered_ngrams',
                                               udf_filter_pos_combs(F.col('finished_ngrams'),
                                                                    F.col('finished_pos_ngrams')))

Below is what we get after filtering for n-grams.

In [48]:
processed_review.select('filtered_ngrams').limit(5).show(truncate=90)

22/12/02 08:35:00 WARN PythonUDFRunner: Detected deadlock while completing task 6.0 in stage 110 (TID 2940): Attempting to kill Python Worker


22/12/02 08:35:04 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 110 (TID 2934): Attempting to kill Python Worker
22/12/02 08:35:04 WARN PythonUDFRunner: Detected deadlock while completing task 1.0 in stage 110 (TID 2935): Attempting to kill Python Worker


22/12/02 08:35:12 WARN PythonUDFRunner: Detected deadlock while completing task 7.0 in stage 110 (TID 2941): Attempting to kill Python Worker
22/12/02 08:35:12 WARN PythonUDFRunner: Detected deadlock while completing task 5.0 in stage 110 (TID 2939): Attempting to kill Python Worker


22/12/02 08:35:14 WARN PythonUDFRunner: Detected deadlock while completing task 2.0 in stage 110 (TID 2936): Attempting to kill Python Worker


22/12/02 08:35:18 WARN PythonUDFRunner: Detected deadlock while completing task 3.0 in stage 110 (TID 2937): Attempting to kill Python Worker


22/12/02 08:35:24 WARN PythonUDFRunner: Detected deadlock while completing task 4.0 in stage 110 (TID 2938): Attempting to kill Python Worker


22/12/02 08:37:51 WARN PythonUDFRunner: Detected deadlock while completing task 8.0 in stage 110 (TID 2942): Attempting to kill Python Worker
+------------------------------------------------------------------------------------------+
|                                                                           filtered_ngrams|
+------------------------------------------------------------------------------------------+
|[relax_conversation, book_dontmesswithasilverback, dontmesswithasilverback_govegan, gov...|
|[sweet_potato, courgette_fritter, mint_yoghurt, yoghurt_dip, dip_vegan, vegan_plant, pl...|
|[creamy_mushroom, mushroom_bucatini, bucatini_vegan, vegan_veganforlife, veganforlife_f...|
|[polypieter_veggie, super_vegan, get_deficiency, be_vegan, theory_hey, dont_read, produ...|
|[natural_origin, origin_ingredient, vegan_society, love_wink, wink_fragrance, fragrance...|
+------------------------------------------------------------------------------------------+



Now we have unigrams and n-grams stored in different columns in the dataframe. Let's combine them together.

In [49]:
from pyspark.sql.functions import concat

processed_review = processed_review.withColumn('final', 
                                               concat(F.col('filtered_unigrams'), 
                                                      F.col('filtered_ngrams')))

And this is our final look of the data.

In [50]:
processed_review.select('final').limit(5).show(truncate=90)

22/12/02 08:40:09 WARN PythonUDFRunner: Detected deadlock while completing task 6.0 in stage 127 (TID 3102): Attempting to kill Python Worker


22/12/02 08:40:15 WARN PythonUDFRunner: Detected deadlock while completing task 4.0 in stage 127 (TID 3100): Attempting to kill Python Worker
22/12/02 08:40:15 WARN PythonUDFRunner: Detected deadlock while completing task 3.0 in stage 127 (TID 3099): Attempting to kill Python Worker


22/12/02 08:40:17 WARN PythonUDFRunner: Detected deadlock while completing task 2.0 in stage 127 (TID 3098): Attempting to kill Python Worker


22/12/02 08:40:19 WARN PythonUDFRunner: Detected deadlock while completing task 7.0 in stage 127 (TID 3103): Attempting to kill Python Worker
22/12/02 08:40:19 WARN PythonUDFRunner: Detected deadlock while completing task 5.0 in stage 127 (TID 3101): Attempting to kill Python Worker
22/12/02 08:40:19 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 127 (TID 3096): Attempting to kill Python Worker


22/12/02 08:40:21 WARN PythonUDFRunner: Detected deadlock while completing task 1.0 in stage 127 (TID 3097): Attempting to kill Python Worker


22/12/02 08:42:44 WARN PythonUDFRunner: Detected deadlock while completing task 8.0 in stage 127 (TID 3104): Attempting to kill Python Worker
+------------------------------------------------------------------------------------------+
|                                                                                     final|
+------------------------------------------------------------------------------------------+
|[livestreaming, conversation, asefrid, book, govegan, livelikeagorilla, veganfortheanim...|
|[sweet, potato, fritter, mint, dip, vegan, plant, base, sweet_potato, courgette_fritter...|
|[creamy, mushroom, bucatini, vegan, veganforlife, forevervegan, creamy_mushroom, mushro...|
|[polypieter, veggie, already, vegan, almost, impossible, get, hey, dont, ingredient, ve...|
|[enrich, origin, ingredient, certify, vegan, wink, fragrance, natural_origin, origin_in...|
+------------------------------------------------------------------------------------------+



## 3.2 Vectorization

Now we are set to vectorization of our data. First, we will proceed with TF (term frequency) vectorization with CountVectorizer in PySpark. We fit tf dictionary and then transform the data to vectors of counts.

Convert a collection of text documents to a matrix of token counts.

In [53]:
from pyspark.ml.feature import CountVectorizer

tfizer = CountVectorizer(inputCol='final', outputCol='tf_features')
tf_model = tfizer.fit(processed_review)
tf_result = tf_model.transform(processed_review)

22/12/02 09:48:13 ERROR Executor: Exception in task 6.0 in stage 170.0 (TID 3586)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/var/folders/x2/hymzscwj7pg_t42hpfwjm20m0000gn/T/ipykernel_6194/2024449930.py", line 9, in translate
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/googletrans/client.py", line 210, in translate
    data, response = self._translate(text, dest, src, kwargs)
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/googletrans/client.py", line 108, in _translate
    r = self.client.get(url, params=params)
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/httpx/_client.py", line 755, in get
    return self.request(
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/httpx/_client.py", line 600, in request
    return self.send(
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/httpx/_client.py", line 620, in send
    response 

Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 730, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/var/folders/x2/hymzscwj7pg_t42hpfwjm20m0000gn/T/ipykernel_6194/2024449930.py", line 9, in translate
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/googletrans/client.py", line 210, in translate
    data, response = self._translate(text, dest, src, kwargs)
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/googletrans/client.py", line 108, in _translate
    r = self.client.get(url, params=params)
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/httpx/_client.py", line 755, in get
    return self.request(
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/httpx/_client.py", line 600, in request
    return self.send(
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/httpx/_client.py", line 620, in send
    response = self.send_handling_redi

22/12/02 09:48:13 WARN TaskSetManager: Lost task 8.0 in stage 170.0 (TID 3588) (192.168.0.134 executor driver): TaskKilled (Stage cancelled)


After we get TF results, we can account for words that are frequent for all the documents. We can use IDF (inverse document frequency) to lower score of such words.

The inverse document frequency is a measure of whether a term is common or rare in a given document corpus. It is obtained by dividing the total number of documents by the number of documents containing the term in the corpus.

While computing TF, all terms are considered equally important. However it is known that certain terms, such as “is”, “of”, and “that”, may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing IDF, an inverse document frequency factor is incorporated which diminishes the weight of terms that occur very frequently in the document set and increases the weight of terms that occur rarely.
IDF is the inverse of the document frequency which measures the informativeness of term t. When we calculate IDF, it will be very low for the most occurring words such as stop words (because stop words such as “is” is present in almost all of the documents, and N/df will give a very low value to that word). This finally gives what we want, a relative weightage.

Now there are few other problems with the IDF , in case of a large corpus,say 100,000,000 , the IDF value explodes , to avoid the effect we take the log of idf .


https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Ftf-term-frequency-idf-inverse-document-frequency-from-scratch-in-python-6c2b61b78558&psig=AOvVaw2NphvHAPexM_UP_4UAcgLP&ust=1670086684586000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCJjOxO6z2_sCFQAAAAAdAAAAABAE

In [ ]:
from pyspark.ml.feature import IDF

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

22/12/01 13:26:03 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


22/12/01 13:32:45 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


## LDA

https://towardsdatascience.com/latent-dirichlet-allocation-lda-9d1cd064ffa2

Finally, we are ready to model topics in our data with LDA (Latent Dirichlet Allocation). To use the algorithm, we have to provide the number of topics we presume our data contains and the number of iterations for the LDA algorithm. Then, we initialize the model and train it.

In [ ]:
tfidf_result.show()

22/12/01 13:33:18 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


22/12/01 13:38:47 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 175 (TID 3601): Attempting to kill Python Worker
22/12/01 13:38:47 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 175 (TID 3601): Attempting to kill Python Worker
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                name|    screen_name|          created_at|           full_text|    post_created_at|                text|         translation|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|   filtered_unigrams|     filtered_ngrams|               final|         tf_features|     tf_idf_features|
+--------------------+---------------+--------------

In [ ]:
from pyspark.ml.clustering import LDA

num_topics = 3
max_iter = 10

lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features').setTopicDistributionCol("topicDistributionCol")
lda_model = lda.fit(tfidf_result)
transformed = lda_model.transform(tfidf_result)

22/12/01 14:05:35 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


ERROR:root:KeyboardInterrupt while sending command.>                (0 + 0) / 9]
Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
transformed.show()

22/12/01 13:47:07 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                name|    screen_name|          created_at|           full_text|    post_created_at|                text|         translation|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|   filtered_unigrams|     filtered_ngrams|               final|         tf_features|     tf_idf_features|topicDistributionCol|
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----

Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 730, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74

In [ ]:
def get_topic(input_list):
    max = input_list[0]
    index = 0
    for i in range(1,len(input_list)):
        if input_list[i] > max:
            max = input_list[i]
            index = i
    return index

get_topic_udf = udf(lambda z: get_topic(z), IntegerType())

In [ ]:
transformed = transformed.withColumn('topic', get_topic_udf("topicDistributionCol"))

In [ ]:
freq_month = transformed.withColumn("year", year(df["post_created_at"]))
freq_month = freq_month.withColumn("month", month(df["post_created_at"]))

freq_month = freq_month.groupBy('year', 'month', 'topic').agg(countDistinct("full_text"))\
               .withColumnRenamed("count(full_text)", "freq") \
                    .sort('year', 'month', ascending = True)
freq_month = freq_month.select(concat_ws('_',freq_month.year, freq_month.month)\
                            .alias('date'), 'topic', 'freq').toPandas()


22/12/01 13:52:29 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


22/12/01 14:01:04 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


22/12/01 14:01:05 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


22/12/01 14:01:05 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/12/01 14:01:06 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


In [ ]:
freq_month

,date,topic,freq
0,2021_10,5,9472
1,2021_10,4,8047
2,2021_10,2,4126
3,2021_10,1,4224
4,2021_10,3,3156
...,...,...,...
73,2022_10,3,540
74,2022_10,2,531
75,2022_10,0,395
76,2022_10,5,825


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                   x=freq_month["date"],
                   y=freq_month["topic"],
                   z=freq_month["freq"],
                   hoverongaps = True))
fig.show()

In [ ]:
freq_month

,date,topic,freq
0,2021_10,5,9472
1,2021_10,4,8047
2,2021_10,2,4126
3,2021_10,1,4224
4,2021_10,3,3156
...,...,...,...
73,2022_10,3,540
74,2022_10,2,531
75,2022_10,0,395
76,2022_10,5,825


In [ ]:
print((transformed.count(), len(transformed.columns)))

(592251, 18)


To be able to see words that characterize the defined topics, we need to convert word ids into actual words with the custom function. This function will again be converted to PySpark UDF to be used on our topic dataframe.

In [ ]:
vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

Let's define the number of top words per topic we would like to see and extract the words with our function.

In [ ]:
num_top_words = 10

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=90)

+-----+------------------------------------------------------------------------------------------+
|topic|                                                                                topicWords|
+-----+------------------------------------------------------------------------------------------+
|    0|                                   [vegan, be_vegan, im, eat, een, en, food, ik, day, die]|
|    1|                                      [ich, und, que, das, ist, nicht, die, les, soap, es]|
|    2|[vegan, diet, raw, twitter, atheist, healthy, instagram_facebook, twitter_instagram, in...|
|    3|              [recipe, vegan, amp, soap, skincare, organic, check, check_mark, mark, food]|
|    4|                     [vegan, amp, cheese, im, make, food, sob, recipe, vegetarian, burger]|
|    5|              [heart, rofl, vegan, be_vegan, go, food, collision, rofl_rofl, animal, fire]|
+-----+------------------------------------------------------------------------------------------+



In [ ]:
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

topic: 0
*************************
vegan
be_vegan
im
eat
een
en
food
ik
day
die
*************************
topic: 1
*************************
ich
und
que
das
ist
nicht
die
les
soap
es
*************************
topic: 2
*************************
vegan
diet
raw
twitter
atheist
healthy
instagram_facebook
twitter_instagram
instagram_facebook_youtube
visit
*************************
topic: 3
*************************
recipe
vegan
amp
soap
skincare
organic
check
check_mark
mark
food
*************************
topic: 4
*************************
vegan
amp
cheese
im
make
food
sob
recipe
vegetarian
burger
*************************
topic: 5
*************************
heart
rofl
vegan
be_vegan
go
food
collision
rofl_rofl
animal
fire
*************************


## 3.3 Visualization